In [17]:
# Import libraries
import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px

In [18]:
# Load data
file_name = "stats_evenements_2025-04-17_11-25-16.csv"
df = pd.read_csv(file_name, encoding="ISO-8859-1", sep=";")

In [19]:
# Convert Date Événement to datetime
df["Date Événement"] = pd.to_datetime(df["Date Événement"])

# Extract month and year from the 'Date Événement' column and create a new columns 'Month' and 'Year'
df["month"] = df["Date Événement"].dt.month
df["year"] = df["Date Événement"].dt.year
df["date"] = df["Date Événement"].dt.date

# Map month numbers to month names
month_names = {
    1: "Janvier",
    2: "Février",
    3: "Mars",
    4: "Avril",
    5: "Mai",
    6: "Juin",
    7: "Juillet",
    8: "Août",
    9: "Septembre",
    10: "Octobre",
    11: "Novembre",
    12: "Décembre",
}

df["month_names"] = df["month"].map(month_names)

In [20]:
# Select a year
year_list = list(df.year.unique())

selected_year = st.selectbox("Sélectionner une année", year_list)
df_selected_year = df[df.year == selected_year]

2025-04-24 09:56:52.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 09:56:52.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 09:56:52.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 09:56:52.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 09:56:52.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 09:56:52.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [21]:
df_temp = df_selected_year.loc[
    df_selected_year["Libellé Événement"] == "Publication offre d'emploi"
]

grouped = (
    df_temp.groupby(["month", "month_names", "Code Opérateur"])
    .size()
    .reset_index(name="count")
)

fig = px.bar(
    grouped,
    x="month",
    y="count",
    color="Code Opérateur",
    barmode="group",
    labels={
        "month_names": "Mois",
        "count": "Nombre d'annonces",
        "Code Opérateur": "Opérateur",
    },
)
fig.update_layout(xaxis_title="Mois", yaxis_title=None, xaxis=dict(tickmode="linear"))

fig.show()

In [ ]:
# Date de début de publication
df_temp = df_selected_year.loc[df_selected_year["Libellé Événement"] == "Publication offre d'emploi"]
df_temp["debut_publication"] = pd.to_datetime(df_temp["Date Événement"])

# Date de fin de publication
df_temp2 = df_selected_year.loc[df_selected_year["Libellé Événement"] == "Dépublication offre d'emploi"]
df_temp2["fin_publication"] = pd.to_datetime(df_temp2["Date Événement"])

# Ensure 'Code Opérateur' exists in both DataFrames before merging
if "Code Opérateur" in df_temp.columns and "Code Opérateur" in df_temp2.columns:
	# Merge the two DataFrames on 'Id Offre' to align the publication and depublication dates
	merged_df = pd.merge(
		df_temp[["Code Opérateur", "Id Offre", "debut_publication"]],
		df_temp2[["Code Opérateur", "Id Offre", "fin_publication"]],
		on=["Id Offre", "Code Opérateur"],  # Merge on both 'Id Offre' and 'Code Opérateur'
		how="inner"
	)
else:
	raise KeyError("The column 'Code Opérateur' is missing in one of the DataFrames.")

# Calculate the duration of publication
merged_df["duree_publication"] = merged_df["fin_publication"] - merged_df["debut_publication"]
print(merged_df[["Code Opérateur", "duree_publication"]])

# calculate the average duration of publication for each operator
avg_duration = merged_df.groupby("Code Opérateur")["duree_publication"].mean().reset_index()
print(avg_duration)
# avg_duration["duree_publication"] = avg_duration["duree_publication"].dt.days
# avg_duration["duree_publication"] = avg_duration["duree_publication"].astype(int)


    Code Opérateur       duree_publication
0           RATPLY 17 days 23:59:52.133572
1               RU  2 days 00:00:00.849193
2           RATPLY 25 days 23:59:38.534174
3           RATPLY  0 days 10:00:04.047650
4           RATPLY 27 days 23:59:42.527675
..             ...                     ...
111          KEOLY  3 days 22:57:28.819673
112          KEOLY  3 days 22:57:29.357056
113          KEOLY  0 days 13:59:52.868615
114          KEOLY  0 days 13:59:52.980880
115         RATPLY  1 days 00:00:07.099392

[116 rows x 2 columns]
  Code Opérateur          duree_publication
0          KEOLY 15 days 00:29:58.227263038
1         RATPLY 26 days 23:25:39.010685500
2             RU 11 days 00:12:22.071441937
  Code Opérateur  duree_publication
0          KEOLY                 15
1         RATPLY                 26
2             RU                 11


C:\Users\VBO\AppData\Local\Temp\ipykernel_25744\1840436963.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\VBO\AppData\Local\Temp\ipykernel_25744\1840436963.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

#Typologie des offres
# Sélectionner uniquement les publications d'offre
df_temp = df_selected_year.loc[
    df_selected_year["Libellé Événement"] == "Publication offre d'emploi"
]

# Sélectionner les colonnes d'intérêt
df_temp = df_temp[["Code Opérateur", "Id Offre", "Date Événement", "Titre Offre"]]

# Tout passer en minuscule
df_temp["Titre Offre"] = df_temp["Titre Offre"].str.lower()

# Supprimer les mentions H/F, F/H, H/F et (H/F) dans le titre de l'offre
df_temp["Titre Offre"] = df_temp["Titre Offre"].str.replace(r"\s*\(H/F\)|\s*H/F|\s*\(F/H\)|\s*F/H", "", regex=True, case=False)

# Supprimer les mentions .e dans le titre de l'offre
df_temp["Titre Offre"] = df_temp["Titre Offre"].str.replace(r"\s*\.e", "", regex=True, case=False)

# Remplacer é par e
df_temp["Titre Offre"] = df_temp["Titre Offre"].str.replace(r"\s*é", "e", regex=True, case=False)


nltk.download('stopwords')
french_stop_words = stopwords.words('french')

vectorizer = TfidfVectorizer(stop_words=french_stop_words)
X = vectorizer.fit_transform(df_temp['Titre Offre'])

# Application de K-means pour le clustering
num_clusters = 10  # Vous pouvez ajuster le nombre de clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_temp['Cluster'] = kmeans.fit_predict(X)

# Réduction de dimensionnalité pour visualisation
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.toarray())

# Visualisation des clusters avec Plotly
import plotly.express as px

fig = px.scatter(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color=df_temp['Cluster'].astype(str),
    labels={'x': 'PCA Component 1', 'y': 'PCA Component 2', 'color': 'Cluster'},
    title="Visualisation des clusters de titres d'emploi",
    color_discrete_sequence=px.colors.qualitative.Plotly  # Use a predefined color set
)
fig.show()

# Affichage des premières lignes avec les clusters
print(df_temp.head())

# Sauvegarde du fichier mis à jour
df_temp.to_excel('bricolage2_clustered.xlsx', index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VBO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


    Code Opérateur  Id Offre             Date Événement  \
81          RATPLY       305 2025-04-15 18:05:42.480849   
83          RATPLY       304 2025-04-15 18:05:42.349793   
134         RATPLY       303 2025-04-14 18:05:31.734589   
136         RATPLY       302 2025-04-14 18:05:31.634199   
138          KEOLY       301 2025-04-14 18:00:30.210751   

                                           Titre Offre  Cluster  
81                                   responsable achat        3  
83   technicien contrôle intervention dci ml et sia...        1  
134      responsable formation et securite ferroviaire        0  
136          chef d?equipe maintenance metro / tramway        1  
138                              responsable de lignes        3  


In [70]:
# Mapping des Tenant ID vers les courriels
tenant_courriel_mapping = {
    'b87cc266-09c4-40cc-8dfa-c92e08bf9cb4': 'anonyme@ratpdev.com',
    '7124e463-2734-41bf-bddb-3e475374f94c': 'anonyme@keolis-lyon.fr',
    '6b23e274-b621-4690-a6c9-bf8828efd33e': 'anonyme@mobilites-lyonnaises.fr',
    '846ece69-a6d3-4892-83ca-a966df6f640e': 'anonyme@sytral.fr',
    'fa41821e-c12a-4f5a-9a6c-8a85b6d803bc': 'admin@admin'
}

# Créer une nouvelle colonne 'Courriel' dans le DataFrame df_temp
df_temp = df_selected_year.loc[df_selected_year["Libellé Événement"] == "Candidature"]
df_temp['courriel'] = df_temp['Tenant ID'].map(tenant_courriel_mapping)

# Count grouped by "Code Opérateur" and "courriel"
grouped = df_temp.groupby(["Code Opérateur", "courriel"]).size().reset_index(name="count")

# Create a bar plot
fig = px.bar(
    grouped,
    x="courriel",
    y="count",
    color="Code Opérateur",
    barmode="group",
    labels={
        "courriel": "Courriel",
        "count": "Nombre de candidatures",
        "Code Opérateur": "Opérateur",
    },
)

fig.update_layout(
    xaxis_title="Courriel",
    yaxis_title="Nombre de candidatures",
    xaxis=dict(tickmode="linear"),
    title="Répartition des candidatures par courriel et opérateur"
)

fig.show()


C:\Users\VBO\AppData\Local\Temp\ipykernel_25744\447688424.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

